In [ ]:
import numpy as np
import pandas as pd
import datetime
import math
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [ ]:
data = pd.read_csv('/content/final_dataset.csv', index_col = False)
data.drop(['Unnamed: 0'], axis=1, inplace= True)

In [ ]:
data.shape

(1482640, 32)

### Data precessing for adapting the model :

In [ ]:
# Transform categorical data to numerical data : ['weather_main', 'weather_description']
categorical_features = ['weather_main', 'weather_description']
one_hot_encode = pd.concat([pd.get_dummies(data[categorical_features], drop_first=True, prefix=col) for col in categorical_features], axis=1)
one_hot_encode.head()

,weather_main_Clouds,weather_main_Rain,weather_main_Snow,weather_main_few clouds,weather_main_heavy intensity rain,weather_main_light rain,weather_main_light snow,weather_main_moderate rain,weather_main_overcast clouds,weather_main_scattered clouds,...,weather_description_Snow,weather_description_few clouds,weather_description_heavy intensity rain,weather_description_light rain,weather_description_light snow,weather_description_moderate rain,weather_description_overcast clouds,weather_description_scattered clouds,weather_description_sky is clear,weather_description_snow
0,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
4,0,1,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
# Drop repeated features :
data = data.drop(['weather_main', 'weather_description'], axis=1).join(one_hot_encode)

In [ ]:
# Transform str to datetimes :
data['WORK_DATE'] = pd.to_datetime(data['WORK_DATE'], format = '%Y-%m-%d')
data['DEB_TIME'] = pd.to_datetime(data['DEB_TIME'], format = '%Y-%m-%d')
data['FIN_TIME'] = pd.to_datetime(data['FIN_TIME'], format = '%Y-%m-%d')

In [ ]:
# Create Time series Features:
data['day'] = data['WORK_DATE'].dt.day
data['month'] = data['WORK_DATE'].dt.month
data['year'] = data['WORK_DATE'].dt.year
data['DEB_TIME_MIN'] = data['DEB_TIME'].dt.minute
data['FIN_TIME_MIN'] = data['FIN_TIME'].dt.minute

In [ ]:
# split training data and testing data : select the last 6 months data as testing data
start_date = datetime.datetime(2022, 2, 1)
end_date = datetime.datetime(2022, 8, 1)
data_train = data[data['WORK_DATE'] < start_date]
data_test = data[(data['WORK_DATE'] <= end_date) & (data['WORK_DATE'] >= start_date)]

data_train.drop(['WORK_DATE', 'DEB_TIME', 'FIN_TIME'], axis=1, inplace=True)
data_test.drop(['WORK_DATE', 'DEB_TIME', 'FIN_TIME'], axis=1, inplace=True)
data_train.shape, data_test.shape

<ipython-input-11-5d355cfc5188>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train.drop(['WORK_DATE', 'DEB_TIME', 'FIN_TIME'], axis=1, inplace=True)
<ipython-input-11-5d355cfc5188>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_test.drop(['WORK_DATE', 'DEB_TIME', 'FIN_TIME'], axis=1, inplace=True)


((1255049, 56), (227591, 56))

In [ ]:
# Grid search :
# A parameter grid for XGBoost
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'learning_rate': [.03, 0.05, 0.07]
        }


In [ ]:
# Train a XGBoost model for each entity :
def xgboost_forecast(entity, data_train, data_test, params):
  '''
  entity : string of the name of the desired entity
  data_train: dataframe, complete train data
  data_test : dataframe, complete test data
  params : dictionary, parameters for grid search
  '''
  # Data preparation :
  train_entity = data_train[data_train['ENTITY_DESCRIPTION_SHORT'] == entity]
  train_entity_X = train_entity.drop(['WAIT_TIME_MAX', 'ENTITY_DESCRIPTION_SHORT'], axis=1)
  train_entity_y = train_entity['WAIT_TIME_MAX']

  test_entity = data_test[data_test['ENTITY_DESCRIPTION_SHORT'] == entity]
  test_entity_X = test_entity.drop(['WAIT_TIME_MAX', 'ENTITY_DESCRIPTION_SHORT'], axis=1)
  test_entity_y = test_entity['WAIT_TIME_MAX']
  # Create a model then random search for best params :
  model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=1000, enable_categorical=True)

  #grid_search = GridSearchCV(model, params)
  #grid_search.fit(train_entity_X , train_entity_y)
  model.fit(train_entity_X , train_entity_y)

  # Model predicting :
  y_pred = model.predict(test_entity_X)

  # Metric calculation:
  mse = np.square(np.subtract(test_entity_y, y_pred)).mean()
  rmse = math.sqrt(mse)
  return y_pred, rmse

In [ ]:
# Output predictions for all entities:
count = 0

result_y = pd.DataFrame()
result_rmse =  {}
for entity in list(data['ENTITY_DESCRIPTION_SHORT'].unique()):

  y_pred, rmse = xgboost_forecast(entity, data_train, data_test, params)

  # Read in results of prediction and rmse :
  test_entity = data[(data['WORK_DATE'] <= end_date) & (data['WORK_DATE'] >= start_date) & (data['ENTITY_DESCRIPTION_SHORT'] == entity)]
  test_entity['prediction'] = y_pred
  test_entity.sort_values(by = ['DEB_TIME'], inplace=True)

  new_df = test_entity[['ENTITY_DESCRIPTION_SHORT', 'DEB_TIME', 'prediction', 'WAIT_TIME_MAX']]
  new_df['prediction'] = new_df['prediction'].apply(lambda x: 0 if x<0 else x) # Turn all negative predictions to 0

  # Concat new dataframe to the result_y dataframe
  result_y = pd.concat([result_y, new_df], ignore_index=True)
  result_rmse[entity] = rmse

  count += 1
  print(count)




<ipython-input-71-3f387a9443b8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity['prediction'] = y_pred
<ipython-input-71-3f387a9443b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity.sort_values(by = ['DEB_TIME'], inplace=True)
<ipython-input-71-3f387a9443b8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

1


<ipython-input-71-3f387a9443b8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity['prediction'] = y_pred
<ipython-input-71-3f387a9443b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity.sort_values(by = ['DEB_TIME'], inplace=True)
<ipython-input-71-3f387a9443b8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

2


<ipython-input-71-3f387a9443b8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity['prediction'] = y_pred
<ipython-input-71-3f387a9443b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity.sort_values(by = ['DEB_TIME'], inplace=True)
<ipython-input-71-3f387a9443b8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

...

<ipython-input-71-3f387a9443b8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity['prediction'] = y_pred
<ipython-input-71-3f387a9443b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity.sort_values(by = ['DEB_TIME'], inplace=True)
<ipython-input-71-3f387a9443b8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

24


<ipython-input-71-3f387a9443b8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity['prediction'] = y_pred
<ipython-input-71-3f387a9443b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity.sort_values(by = ['DEB_TIME'], inplace=True)
<ipython-input-71-3f387a9443b8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

25
26


<ipython-input-71-3f387a9443b8>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity['prediction'] = y_pred
<ipython-input-71-3f387a9443b8>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_entity.sort_values(by = ['DEB_TIME'], inplace=True)
<ipython-input-71-3f387a9443b8>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

In [ ]:
result_y

,ENTITY_DESCRIPTION_SHORT,DEB_TIME,prediction,WAIT_TIME_MAX
0,Roller Coaster,2022-02-01 09:00:00,0.000000,0
1,Roller Coaster,2022-02-01 09:15:00,0.000000,0
2,Roller Coaster,2022-02-01 09:30:00,0.000000,0
3,Roller Coaster,2022-02-01 09:45:00,1.608427,0
4,Roller Coaster,2022-02-01 10:00:00,19.924141,20
...,...,...,...,...
227586,Vertical Drop,2022-07-26 21:45:00,0.000000,10
227587,Vertical Drop,2022-07-26 22:00:00,0.000000,10
227588,Vertical Drop,2022-07-26 22:15:00,0.000000,10
227589,Vertical Drop,2022-07-26 22:30:00,0.000000,10


In [ ]:
# Save the result of prediction in a .csv file :
result_y.to_csv('Prediction_result_6months.csv')